# Import necessary libraries

In [ ]:
import numpy as np
import joblib
from ipywidgets import Layout
import ipywidgets as widgets
from IPython.display import display

# Load the trained model and scaler


In [ ]:
model = joblib.load('/content/xgb_model.pkl')  # XGBoost trained model
scaler = joblib.load('scaler.pkl')  # StandardScaler used for input/output scaling

In [ ]:
# Display model details for debugging (Optional)
# print("Model type:", type(model))
# print("Expected number of features:", model.n_features_in_)

#GUI for Prediction

In [ ]:
# Create input fields for user input
wide_layout = Layout(width='350px')  # Adjust width as needed
label_style = {'description_width': '180px'}  # Adjust as needed


tf = widgets.FloatText(description="Flange Thickness:", layout=wide_layout, style=label_style)
tw = widgets.FloatText(description="Web Thickness:", layout=wide_layout, style=label_style)
bf = widgets.FloatText(description="Flange Width:", layout=wide_layout, style=label_style)
#hw = widgets.FloatText(description="Web Height:", layout=wide_layout, style=label_style)
fyw = widgets.FloatText(description="Web Yield Strength:", layout=wide_layout, style=label_style)
fyf = widgets.FloatText(description="Flange Yield Strength:", layout=wide_layout, style=label_style)
b = widgets.FloatText(description="Tension Field Width:", layout=wide_layout, style=label_style)
L = widgets.FloatText(description="Moment Span Length:", layout=wide_layout, style=label_style)
d = widgets.FloatText(description="Flange Depth:", layout=wide_layout, style=label_style)

# Output display field
output_label = widgets.Label(value="Prediction: ")

# Function to process user input, scale it, predict, and unscale the output
def predict_model(button):
    try:
        # Retrieve user inputs and convert them to floats
        tf_val = float(tf.value) if tf.value else 0
        tw_val = float(tw.value) if tw.value else 0
        bf_val = float(bf.value) if bf.value else 0
        #hw_val = float(hw.value) if hw.value else 0
        fyw_val = float(fyw.value) if fyw.value else 0
        fyf_val = float(fyf.value) if fyf.value else 0
        b_val = float(b.value) if b.value else 0
        L_val = float(L.value) if L.value else 0
        d_val = float(d.value) if d.value else 0

        # Debugging: Print parsed values
        #print("Parsed values:", tf_val, tw_val, bf_val, hw_val, fyw_val, fyf_val, b_val, L_val)

        # Prevent division by zero errors
        if tw_val == 0 or tf_val == 0 or b_val == 0:
            output_label.value = "Error: Inputs cannot be zero."
            return

        # Compute derived feature ratios for the model
        tf_tw = tf_val / tw_val
        bf_2tf = bf_val / (2 * tf_val)
        hw_tw = d_val / tw_val
        fyw_fyf = fyw_val / fyf_val
        bf_hw = bf_val / d_val
        b_d = b_val / d_val
        L_b = L_val / b_val

        # Create input feature array
        inputs = np.array([[tf_tw, bf_2tf, hw_tw, fyw_fyf, bf_hw, b_d, L_b]])

        # Add a placeholder column (0) to match the 8-column scaler shape
        inputs_with_placeholder = np.hstack((inputs, np.zeros((inputs.shape[0], 1))))

        # Scale the inputs using the saved scaler
        inputs_scaled = scaler.transform(inputs_with_placeholder)

        # Remove the placeholder column before passing to the model
        inputs_scaled = inputs_scaled[:, :-1]

        # Predict using the trained model (scaled output)
        y_pred_scaled = model.predict(inputs_scaled)

        # Create a dummy array to hold the scaled output for inverse transformation
        dummy_array = np.zeros((y_pred_scaled.shape[0], 8))  # 8 columns (7 input + 1 output)
        dummy_array[:, -1] = y_pred_scaled  # Place prediction in last column

        # Inverse transform to get the actual output
        y_pred_unscaled = scaler.inverse_transform(dummy_array)[:, -1]

        # Display the final unscaled prediction in kN
        output_label.value = f"Prediction: {y_pred_unscaled[0]:.2f} kN"

    except ValueError as ve:
        output_label.value = f"Value Error: Invalid numeric values."
        print("Error details:", ve)

    except Exception as e:
        output_label.value = f"Unexpected error: {str(e)}"
        print("Unexpected error:", e)

# Create Predict button
predict_button = widgets.Button(description="Predict", button_style='success')
predict_button.on_click(predict_model)

# Display all widgets
display(tf, tw, bf, fyw, fyf, b, d, L, predict_button, output_label)


FloatText(value=0.0, description='Flange Thickness:', layout=Layout(width='350px'), style=DescriptionStyle(des…

FloatText(value=0.0, description='Web Thickness:', layout=Layout(width='350px'), style=DescriptionStyle(descri…

FloatText(value=0.0, description='Flange Width:', layout=Layout(width='350px'), style=DescriptionStyle(descrip…

FloatText(value=0.0, description='Web Yield Strength:', layout=Layout(width='350px'), style=DescriptionStyle(d…

FloatText(value=0.0, description='Flange Yield Strength:', layout=Layout(width='350px'), style=DescriptionStyl…

FloatText(value=0.0, description='Tension Field Width:', layout=Layout(width='350px'), style=DescriptionStyle(…

FloatText(value=0.0, description='Flange Depth:', layout=Layout(width='350px'), style=DescriptionStyle(descrip…

FloatText(value=0.0, description='Moment Span Length:', layout=Layout(width='350px'), style=DescriptionStyle(d…

Button(button_style='success', description='Predict', style=ButtonStyle())

Label(value='Prediction: ')